In [6]:
import numpy as np
from gurobipy import *
import random

In [60]:
def connected(vertex,adjacency,n,visited):
    visited[vertex] = 1
    row = adjacency[vertex]
    
    for node in range(n):
        if row[node] > 0 and visited[node] < 1:
            visited = connected(node,adjacency,n,visited)
    
    return visited

def rand_DAG(N,p):
    go = True
    while go:
        dag = np.zeros([N,N])
        for i in np.arange(N):
            for j in np.arange(i+1,N):
                if (random.random() < p):
                    dag[i][j] = 1
        visited = np.zeros(N)
        visited = connected(0,dag,N,visited)
        if(visited[N-1] == 1):
            go = False      
    return dag

def solve_TOP(adjacency,n,k,prizes,numPaths):
    nodes = []
    for i in range(n):
        nodes.append(i)
    m = Model("OP2")
    
    m.setParam(GRB.Param.PoolSolutions,numPaths)
    m.setParam(GRB.Param.PoolSearchMode,2)
    
    inflow = np.zeros(n)
    inflow[0] = k
    inflow[n-1] = -k
    
    visited = m.addVars(nodes, vtype=GRB.BINARY,  name="visited")
    flows = m.addVars(nodes,nodes, vtype = GRB.INTEGER, name="flows")
    
    # Flow conservation
    m.addConstrs(
    (flows.sum('*',j) + inflow[j] == flows.sum(j,'*') for j in nodes), "Conservation")
    
    # Flow only on existing arcs
    m.addConstrs((flows[i,j] <= adjacency[i,j] for i in nodes for j in nodes),"EdgesPresent")
    
    # Visited nodes
    m.addConstrs((visited[j] <= flows.sum('*',j) for j in nodes), "WhetherVisited")
    
#     obj = quicksum(visited[j] * prizes[j])
    
    # Objective Function
    m.setObjective((quicksum(visited[j]*prizes[j] for j in nodes)), GRB.MAXIMIZE)
    
    m.optimize()
    if m.status == GRB.Status.OPTIMAL:
        print(m.status)
        solution = m.getAttr('x',flows)
        for i in nodes:
            for j in nodes:
                if solution[i,j] > 0:
                    print('%s -> %s: %g' % (i, j, solution[i,j]))
    
    #save model
    
    return m,flows

# Assumes k = 1, constructs path from solution
def construct_path(m,n,flows):
    solution = m.getAttr('xn',flows)
    path = []
    nodes = range(n)
    for i in nodes:
        for j in nodes:
            if solution[i,j] > 0:
                path.append(j)
    return path

def score_paths(paths,k,prizes):
    scores = np.zeros(k,dtype=int)
    prizes_copy = prizes.copy()
    print(scores)
    go = True
    step = -1
    while go:
        go = False
        step += 1
        for player in range(k):
            path = paths[player]
            if len(path) > step:
                scores[player] += prizes_copy[path[step]]
                prizes_copy[path[step]] = 0
                go = True
    return scores


In [61]:
def unreserved_best_response(adjacency,n,prizes,first_path):
    nodes = []
    for i in range(n):
        nodes.append(i)
    m = Model("OP2")
    
    inflow = np.zeros(n)
    inflow[0] = 1
    inflow[n-1] = -1
    
    visited = m.addVars(nodes, vtype=GRB.BINARY,  name="visited")
    flows = m.addVars(nodes,nodes, vtype = GRB.INTEGER, name="flows")
    arrivals2 = m.addVars(nodes, vtype=GRB.INTEGER, name="arrivals2")
#     arrivals1 = m.addVars(nodes, name="arrivals1")
    second_prizes = m.addVars(nodes, vtype = GRB.BINARY, name = "second_prizes")
    
    full_arrivals1 = np.zeros(n)+n
    full_arrivals1[0] = 0
    for i in range(len(first_path)):
        temp = first_path[i]
        full_arrivals1[temp] = i+1
    
        # Flow conservation
    m.addConstrs((flows.sum('*',j) + inflow[j] == flows.sum(j,'*') for j in nodes), "Conservation")
    
    # Flow only on existing arcs
    m.addConstrs((flows[i,j] <= adjacency[i,j] for i in nodes for j in nodes),"EdgesPresent")
    
    # Prizes uncollected if not visited
    m.addConstrs((second_prizes[j] <= flows.sum('*',j) for j in nodes), "WhetherVisited")
    
    # Prizes uncollected if not visited FIRST
    m.addConstrs((second_prizes[i] <= full_arrivals1[i] - arrivals2[i] for i in nodes), "visitedFirst")
    
    # Enforce correct arrival times
    #m.addConstrs((arrivals2[i] >= (arrivals2[j] + 1) * flows(j,i) for i >=1 in nodes for j < i in nodes), "arrivalEnforced")
    m.addConstrs((arrivals2[i] >= arrivals2[j] + (flows[j,i] - 1)* n + flows[j,i] for i in nodes for j in nodes), "arrivalEnforced")
#     m.addConstrs((arrivals2[i] >= arrivals2[j] -1 + 2*flows[j,i] for i in nodes for j in nodes), "arrivalEnforced")
    
    # Set starting point
    m.addConstr(arrivals2[0] == 0, "startEnforced")
    
    m.setObjective((quicksum(second_prizes[j]*prizes[j] for j in nodes)), GRB.MAXIMIZE)
    
    m.optimize()
    if m.status == GRB.Status.OPTIMAL:
        print(m.status)
        solution = m.getAttr('x',flows)
        for i in nodes:
            for j in nodes:
                if solution[i,j] > 0:
                    print('%s -> %s: %g' % (i, j, solution[i,j]))
    
    #save model
    
    return m,flows
    
# Analysis for k best paths (2 players)
# def k_paths(adj,n,prizes,num_best):
#     model, flows = solve_TOP


In [62]:
n = 100
p = 0.3
k = 1
prizes = []
for i in range(n):
    prizes.append(random.randint(1,2))
prizes[0] = 0
prizes[n-1] = 0
adj = rand_DAG(n,p)
print(adj)
print(prizes)

temp = np.array([4,3,2])
print(len(temp))

[[ 0.  0.  0. ...,  1.  0.  1.]
 [ 0.  0.  1. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[0, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 0]
3


In [64]:
model,flows = solve_TOP(adj,n,k,prizes,1);
# temp = model.getAttr('x');
# print(model.getAttr('x'));
# print(construct_path(model,n,flows))
# model.setParam(GRB.Param.SolutionNumber,99)
# print(model.getAttr('xn'))
first_path = construct_path(model,n,flows);
print(first_path)
new_model,new_flows = unreserved_best_response(adj,n,prizes,first_path);
second_path = construct_path(new_model,n,new_flows);
print(second_path)
print(score_paths([first_path,second_path],2,prizes))


Changed value of parameter PoolSolutions to 1
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Optimize a model with 10200 rows, 10100 columns and 39900 nonzeros
Variable types: 0 continuous, 10100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 10034 rows and 8996 columns
Presolve time: 0.03s
Presolved: 166 rows, 1104 columns, 2988 nonzeros
Variable types: 0 continuous, 1104 integer (1104 binary)

Root relaxation: objective 6.400000e+01, 415 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      64.0000000   64.00000  0.00%     -    0s

Explored 0 nodes

In [330]:

model,flows = solve_TOP(adj,n,1,prizes,100)
temp = model.getAttr('x')
print(model.getAttr('x'))
# print(construct_path(model,n,flows))
model.setParam(GRB.Param.SolutionNumber,99)
print(model.getAttr('xn'))
print(construct_path(model,n,flows))

Changed value of parameter PoolSolutions to 100
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Optimize a model with 120 rows, 110 columns and 390 nonzeros
Variable types: 0 continuous, 110 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 105 rows and 77 columns
Presolve time: 0.00s
Presolved: 15 rows, 33 columns, 71 nonzeros
Variable types: 0 continuous, 33 integer (33 binary)
Found heuristic solution: objective 7.0000000

Root relaxation: objective 8.000000e+00, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       8.0000000    8.00000  0.00%     -

In [321]:

x = model.getAttr('x',flows)
y = model.getAttr('x')
print(x)
print(y)


{(0, 0): 0.0, (0, 1): 1.0, (0, 2): 0.0, (0, 3): 0.0, (0, 4): 0.0, (0, 5): -0.0, (0, 6): 0.0, (0, 7): -0.0, (0, 8): -0.0, (0, 9): -0.0, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 1.0, (1, 3): -0.0, (1, 4): 0.0, (1, 5): -0.0, (1, 6): -0.0, (1, 7): -0.0, (1, 8): 0.0, (1, 9): -0.0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0, (2, 3): 0.0, (2, 4): 0.0, (2, 5): 1.0, (2, 6): -0.0, (2, 7): 0.0, (2, 8): 0.0, (2, 9): 0.0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0, (3, 4): 0.0, (3, 5): 0.0, (3, 6): 0.0, (3, 7): 0.0, (3, 8): 0.0, (3, 9): 0.0, (4, 0): 0.0, (4, 1): 0.0, (4, 2): 0.0, (4, 3): 0.0, (4, 4): 0.0, (4, 5): -0.0, (4, 6): -0.0, (4, 7): 0.0, (4, 8): 0.0, (4, 9): 0.0, (5, 0): 0.0, (5, 1): 0.0, (5, 2): 0.0, (5, 3): 0.0, (5, 4): 0.0, (5, 5): 0.0, (5, 6): 1.0, (5, 7): 0.0, (5, 8): -0.0, (5, 9): 0.0, (6, 0): 0.0, (6, 1): 0.0, (6, 2): 0.0, (6, 3): 0.0, (6, 4): 0.0, (6, 5): 0.0, (6, 6): 0.0, (6, 7): 1.0, (6, 8): -0.0, (6, 9): -0.0, (7, 0): 0.0, (7, 1): 0.0, (7, 2): 0.0, (7, 3): 0.0, (7, 4): 0.0, (7, 5): 0